In [1]:
import h5py
import numpy as np
import pandas as pd
import datetime

In [25]:
df=pd.read_csv(filepath_or_buffer=meta_file_name,sep="\t")
type(df["SRR_accession"].values)

numpy.ndarray

NameError: name 'create_dataset' is not defined

In [40]:
np.arange(6)

array([0, 1, 2, 3, 4, 5])

dee2 tsv to srr h5

In [80]:
%%time
import h5py
import numpy as np
import pandas as pd
import datetime
organism_name= 'ecoli'
cache_dir="D://datasets/DEE2/"
h5_file_name = cache_dir+"/srr_files/"+organism_name+ "_SRR_matrix.h5"
meta_file_name = cache_dir+"/metadata/"+organism_name+"_metadata.tsv"
data_file_name = cache_dir +"/raw_files/"+organism_name +"_se.tsv"
with h5py.File(h5_file_name, 'w') as h5_file:
    meta_grp = h5_file.create_group('meta')
    info_grp = h5_file.create_group('info')
    data_grp= h5_file.create_group('data')
    info_grp.create_dataset('version', data="dee2_v1")
    info_grp.create_dataset('creation_date', data="2020-02-23")
    meta_df=pd.read_csv(filepath_or_buffer=meta_file_name,sep="\t")
    dt = h5py.special_dtype(vlen=np.str)
    meta_grp.create_dataset('SRR_accession',data= np.array(meta_df["SRR_accession"], dtype=dt))
    meta_grp.create_dataset('Sample_geo_accession',data=np.array(meta_df["GSE_accession"], dtype=dt))
    meta_grp.create_dataset('Sample_instrument_model',data=np.array(meta_df["Model"], dtype=dt))
    meta_grp.create_dataset('Sample_last_update_date',data=np.array(meta_df["LoadDate"], dtype=dt))
    meta_grp.create_dataset('Sample_library_selection',data=np.array(meta_df["LibrarySelection"], dtype=dt))
    meta_grp.create_dataset('Sample_library_source',data=np.array(meta_df["LibrarySource"], dtype=dt))
    meta_grp.create_dataset('Sample_library_strategy',data=np.array(meta_df["LibraryStrategy"], dtype=dt))
    meta_grp.create_dataset('Sample_organism_ch1',data=np.array(meta_df["ScientificName"], dtype=dt))
    meta_grp.create_dataset('Sample_series_id',data=np.array(meta_df["Sample_name"], dtype=dt))
    meta_grp.create_dataset('Sample_status',data=np.array(meta_df["Consent"], dtype=dt))
    meta_grp.create_dataset('Sample_submission_date',data=np.array(meta_df["ReleaseDate"], dtype=dt))
    meta_grp.create_dataset('Sample_taxid_ch1',data=np.array(meta_df["TaxID"], dtype=dt))
    meta_grp.create_dataset('Sample_quality',data=np.array(meta_df["QC_summary"], dtype=dt))
    meta_grp.create_dataset('Sample_title',data=np.array(meta_df["BioSample"], dtype=dt))
    meta_grp.create_dataset('Sample_type',data=np.full(len(meta_df["BioSample"]),"SRA", dtype=dt))
    h5_file.flush()
    n_srr=len(meta_df["SRR_accession"])
    n_genes=0
    genes= list()
    with open(data_file_name, 'r') as data_file:
        srr, gene_id, gene_count= data_file.readline().split("\t")
        first_srr = srr
        n_genes+=1
        genes.append(gene_id)
        while(True):
            srr, gene_id, gene_count= data_file.readline().split("\t")
            if (srr!=first_srr):
                break
            n_genes+=1
            genes.append(gene_id)
        meta_grp.create_dataset('genes',data=np.array(genes, dtype='S'))
    exp_data=data_grp.create_dataset("expression", (n_srr,n_genes),dtype= 'i4')#, compression="gzip", compression_opts=9)
    h5_file.flush()
    reader= pd.read_csv(data_file_name, sep='\t', chunksize=n_genes,names=["srr","gene","count"],iterator=True)
    srr=0
    chunk_size = 100    
    while(srr<n_srr):
        chunk = reader.get_chunk(chunk_size*n_genes)
        last_ind = (srr+chunk_size) if (srr+chunk_size)<n_srr else n_srr
        exp_data[srr: last_ind,0:n_genes]= chunk["count"].values.reshape(last_ind-srr,n_genes)
        h5_file.flush()      
        srr+=chunk_size
        if(srr%200==0):
            print(srr/n_srr)
print(1)

KeyboardInterrupt: 

from bz2 file

In [10]:
#mklever_dee2 env
import bz2
import h5py
import numpy as np
import pandas as pd
import datetime
organism_name= 'ecoli'
cache_dir="D://datasets/DEE2/"
h5_file_name = cache_dir+"/srr_files/"+organism_name+ "_SRR_matrix.h5"
meta_file_name = cache_dir+"/metadata/"+organism_name+"_metadata.tsv"
data_file_name = cache_dir +"/raw_files/"+organism_name +"_se.tsv.bz2"
with h5py.File(h5_file_name, 'w') as h5_file:
    meta_grp = h5_file.create_group('meta')
    info_grp = h5_file.create_group('info')
    data_grp= h5_file.create_group('data')
    info_grp.create_dataset('version', data="dee2_v1")
    info_grp.create_dataset('creation_date', data="2020-02-23")
    meta_df=pd.read_csv(filepath_or_buffer=meta_file_name,sep="\t")
    dt = h5py.special_dtype(vlen=np.str)
    meta_grp.create_dataset('SRR_accession',data= np.array(meta_df["SRR_accession"], dtype=dt))
    meta_grp.create_dataset('Sample_geo_accession',data=np.array(meta_df["GEO_series"], dtype=dt))
    meta_grp.create_dataset('Sample_instrument_model',data=np.array(meta_df["Model"], dtype=dt))
    meta_grp.create_dataset('Sample_last_update_date',data=np.array(meta_df["LoadDate"], dtype=dt))
    meta_grp.create_dataset('Sample_library_selection',data=np.array(meta_df["LibrarySelection"], dtype=dt))
    meta_grp.create_dataset('Sample_library_source',data=np.array(meta_df["LibrarySource"], dtype=dt))
    meta_grp.create_dataset('Sample_library_strategy',data=np.array(meta_df["LibraryStrategy"], dtype=dt))
    meta_grp.create_dataset('Sample_organism_ch1',data=np.array(meta_df["ScientificName"], dtype=dt))
    meta_grp.create_dataset('Sample_series_id',data=np.array(meta_df["Sample_name"], dtype=dt))
    meta_grp.create_dataset('Sample_status',data=np.array(meta_df["Consent"], dtype=dt))
    meta_grp.create_dataset('Sample_submission_date',data=np.array(meta_df["ReleaseDate"], dtype=dt))
    meta_grp.create_dataset('Sample_taxid_ch1',data=np.array(meta_df["TaxID"], dtype=dt))
    meta_grp.create_dataset('Sample_quality',data=np.array(meta_df["QC_summary"], dtype=dt))
    meta_grp.create_dataset('Sample_title',data=np.array(meta_df["BioSample"], dtype=dt))
    meta_grp.create_dataset('Sample_type',data=np.full(len(meta_df["BioSample"]),"SRA", dtype=dt))
    h5_file.flush()
    n_srr=len(meta_df["SRR_accession"])
    n_genes=0
    genes= list()
    with bz2.BZ2File(data_file_name, 'r') as data_file:
        srr, gene_id, gene_count= data_file.readline().split(b"\t")
        first_srr = srr
        n_genes+=1
        genes.append(gene_id)
        while(True):
            srr, gene_id, gene_count= data_file.readline().split(b"\t")
            if (srr!=first_srr):
                break
            n_genes+=1
            genes.append(gene_id)
        meta_grp.create_dataset('genes',data=np.array(genes, dtype='S'))
    exp_data=data_grp.create_dataset("expression", (n_srr,n_genes),dtype= 'i4')#, compression="gzip", compression_opts=9)
    h5_file.flush()
    reader= pd.read_csv(data_file_name, sep='\t', chunksize=n_genes,names=["srr","gene","count"],iterator=True)
    srr=0
    chunk_size = 100    
    while(srr<n_srr):
        chunk = reader.get_chunk(chunk_size*n_genes)
        last_ind = (srr+chunk_size) if (srr+chunk_size)<n_srr else n_srr
        exp_data[srr: last_ind,0:n_genes]= chunk["count"].values.reshape(last_ind-srr,n_genes)
        h5_file.flush()      
        srr+=chunk_size
        if(srr%200==0):
            print(srr/n_srr)
print(1)

0.02847380410022779
0.05694760820045558
0.08542141230068337
0.11389521640091116
0.14236902050113895
0.17084282460136674
0.19931662870159453
0.22779043280182232
0.25626423690205014
0.2847380410022779
0.3132118451025057
0.3416856492027335
0.3701594533029613
0.39863325740318906
0.4271070615034169
0.45558086560364464
0.48405466970387245
0.5125284738041003
0.541002277904328
0.5694760820045558
0.5979498861047836
0.6264236902050114
0.6548974943052391
0.683371298405467
0.7118451025056948
0.7403189066059226
0.7687927107061503
0.7972665148063781
0.8257403189066059
0.8542141230068337
0.8826879271070615
0.9111617312072893
0.9396355353075171
0.9681093394077449
0.9965831435079726
1


In [95]:
with h5py.File(h5_srr_name, 'r') as h5_srr:
    print(h5_srr["/meta/Sample_geo_accession"][0].startswith(b"GSM"))

False


In [12]:
test=dict()
test.append("1",1)
print(test)

AttributeError: 'dict' object has no attribute 'append'

In [82]:
organism_name= 'ecoli'
cache_dir="D:/datasets/DEE2/"
h5_srr_name = cache_dir+organism_name+ "_SRR_matrix.h5"
h5_gse_name = cache_dir+organism_name+ "_matrix.h5"
gsm_map =dict()
now = datetime.datetime.now()
with h5py.File(h5_srr_name, 'r') as h5_srr:
    with h5py.File(h5_gse_name, 'w') as h5_gse:
        meta_grp = h5_gse.create_group('meta')
        info_grp = h5_gse.create_group('info')
        data_grp= h5_gse.create_group('data')
        info_grp.create_dataset('version', data="dee2_gse_v1")
        info_grp.create_dataset('creation_date', data=now.strftime("%Y.%m.%d"))
        n_genes =len(h5_srr["/meta/genes"])
        meta_grp.create_dataset('genes',data=h5_srr["/meta/genes"])
        n_gsm=len(np.unique([x for x in h5_srr["/meta/Sample_geo_accession"] if ('GSM' in x)]))
        dt = h5py.special_dtype(vlen=np.str)
        meta_grp.create_dataset('Sample_geo_accession',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_instrument_model',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_last_update_date',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_library_selection',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_library_source',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_library_strategy',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_organism_ch1',(n_gsm,),dtype =dt)
        meta_grp.create_dataset('Sample_series_id',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_status',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_submission_date',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_taxid_ch1',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_quality',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_title',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_type',data=np.full(n_gsm,"SRA", dtype =dt))
        exp_data=data_grp.create_dataset("expression", (n_gsm,n_genes),dtype= 'i4')#, compression="gzip", compression_opts=9)
        processed_gse=0
        n_srr=len(h5_srr["/meta/SRR_accession"])
        for i in range(0,n_srr):
            if(i%200==0):
                print(i/n_srr)
            cur_gse=processed_gse
            if (h5_srr["/meta/Sample_geo_accession"][i].startswith("GSM")):
                try:
                    cur_gse=gsm_map[h5_srr["/meta/Sample_geo_accession"][i]]                     
                except KeyError:
                    gsm_map[h5_srr["/meta/Sample_geo_accession"][i]]=processed_gse
                    processed_gse+=1
                exp_data[cur_gse,0:n_genes]+=h5_srr["/data/expression"][i,0:n_genes]
                if (cur_gse==processed_gse-1):
                    meta_grp["Sample_geo_accession"][cur_gse]=h5_srr["/meta/Sample_geo_accession"][i]
                    meta_grp["Sample_instrument_model"][cur_gse]=h5_srr["/meta/Sample_instrument_model"][i]
                    meta_grp["Sample_last_update_date"][cur_gse]=h5_srr["/meta/Sample_last_update_date"][i]
                    meta_grp["Sample_library_selection"][cur_gse]=h5_srr["/meta/Sample_library_selection"][i]
                    meta_grp["Sample_library_source"][cur_gse]=h5_srr["/meta/Sample_library_source"][i]
                    meta_grp["Sample_library_strategy"][cur_gse]=h5_srr["/meta/Sample_library_strategy"][i]
                    meta_grp["Sample_organism_ch1"][cur_gse]=h5_srr["/meta/Sample_organism_ch1"][i]
                    meta_grp["Sample_status"][cur_gse]=h5_srr["/meta/Sample_status"][i]
                    meta_grp["Sample_series_id"][cur_gse]=h5_srr["/meta/Sample_series_id"][i]
                    meta_grp["Sample_submission_date"][cur_gse]=h5_srr["/meta/Sample_submission_date"][i]
                    meta_grp["Sample_quality"][cur_gse]=h5_srr["/meta/Sample_quality"][i]
                    meta_grp["Sample_title"][cur_gse]=h5_srr["/meta/Sample_title"][i]
                    meta_grp["Sample_type"][cur_gse]=h5_srr["/meta/Sample_type"][i]
        h5_gse.flush()





0.0
0.02847380410022779
0.05694760820045558
0.08542141230068337
0.11389521640091116
0.14236902050113895
0.17084282460136674
0.19931662870159453
0.22779043280182232
0.25626423690205014
0.2847380410022779
0.3132118451025057
0.3416856492027335
0.3701594533029613
0.39863325740318906
0.4271070615034169
0.45558086560364464
0.48405466970387245
0.5125284738041003
0.541002277904328
0.5694760820045558
0.5979498861047836
0.6264236902050114
0.6548974943052391
0.683371298405467
0.7118451025056948
0.7403189066059226
0.7687927107061503
0.7972665148063781
0.8257403189066059
0.8542141230068337
0.8826879271070615
0.9111617312072893
0.9396355353075171
0.9681093394077449
0.9965831435079726


srr h5 to gsm h5

In [109]:
%%time
from collections import defaultdict
def unique_fil(srr_group,gse_group,cur_gse,dset,ind):
    cur_vals=srr_group[dset][ind]
    if (len(np.unique(cur_vals))!=1):
        gse_group[dset][cur_gse]=",".join(cur_vals)
    else:
        gse_group[dset][cur_gse]=cur_vals[0]

organism_name= 'ecoli'
cache_dir="D:/datasets/DEE2/"
h5_srr_name = cache_dir+organism_name+ "_SRR_matrix.h5"
h5_gsm_name = cache_dir+organism_name+ "_matrix.h5"
gsm_map =defaultdict(list)
now = datetime.datetime.now()
gsm_list=[]
with h5py.File(h5_srr_name, 'r') as h5_srr:
    with h5py.File(h5_gsm_name, 'w') as h5_gse:
        meta_grp = h5_gsm.create_group('meta')
        info_grp = h5_gsm.create_group('info')
        data_grp= h5_gsm.create_group('data')
        info_grp.create_dataset('version', data="dee2_gse_v1")
        info_grp.create_dataset('creation_date', data=now.strftime("%Y.%m.%d"))
        n_genes =len(h5_srr["/meta/genes"])
        meta_grp.create_dataset('genes',data=h5_srr["/meta/genes"])
        for i_gsm in range(h5_srr["/meta/Sample_geo_accession"].shape[0]):
            row = h5_srr["/meta/Sample_geo_accession"][i_gsm]
            if ('GSM' in row):
                gsm_map[row].append(i_gsm)                      
        n_gsm=len(gsm_map)
        dt = h5py.special_dtype(vlen=np.str)
        meta_grp.create_dataset('Sample_geo_accession',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('SRR_accession',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_instrument_model',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_last_update_date',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_library_selection',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_library_source',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_library_strategy',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_organism_ch1',(n_gsm,),dtype =dt)
        meta_grp.create_dataset('Sample_series_id',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_status',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_submission_date',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_taxid_ch1',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_quality',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_title',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_type',data=np.full(n_gsm,"SRA", dtype =dt))
        exp_data=data_grp.create_dataset("expression", (n_gsm,n_genes),dtype= 'i4')#, compression="gzip", compression_opts=9)
        gsm_list = list(gsm_map.keys())
        for i_gsm in  range(len(gsm_list)):
            cur_gsm=gsm_list[i_gsm]
            meta_grp["Sample_geo_accession"][i_gsm]=cur_gsm
            for key in h5_srr["meta"]:
                if (key == "genes"):
                    continue
                unique_fil(h5_srr["meta"],meta_grp,i_gsm,key,gsm_map[cur_gsm])
            exp_data[i_gsm,0:n_genes]=np.sum(h5_srr["/data/expression"][gsm_map[cur_gsm],0:n_genes],0)
            if(i_gsm%200==0):
                print(i_gsm/len(gsm_list))
        h5_gse.flush()

0.0
0.06263701847792046
0.1252740369558409
0.18791105543376135
0.2505480739116818
0.31318509238960224
0.3758221108675227
0.4384591293454432
0.5010961478233636
0.563733166301284
0.6263701847792045
0.689007203257125
0.7516442217350454
0.8142812402129659
0.8769182586908864
0.9395552771688067
Wall time: 1min 3s


39297
180869
180869


Get sparse matrices for srr and tx transformation

In [52]:
%%time
import bz2
import h5py
import numpy as np
import pandas as pd
import scipy.sparse as sps
from scipy.sparse import csr_matrix
import math
cache_dir="D:/datasets/DEE2/"
organism_name= 'hsapiens'#'hsapiens' 'ecoli'
geo_acc_name = 'Sample_name' # may be "GSE_accession" or "Sample_name"
tx_info_file = cache_dir+"TxInfo/"+ organism_name+"_TxInfo.tsv"
metadata_file = cache_dir+"metadata/"+ organism_name+"_metadata.tsv"

tx_info=pd.read_csv(filepath_or_buffer=tx_info_file,sep="\t")
tx_to_gene= dict(zip(tx_info["TxID"],tx_info["GeneID"]))
tx_ind= {tx:ind for ind,tx in enumerate(tx_to_gene.keys())}
gene_ind= {gene:ind for ind,gene in enumerate(pd.unique(tx_info["GeneID"]))}
tx_to_gene_matrix= sps.lil_matrix((len(tx_ind),len(gene_ind)))
for tx_key in tx_to_gene:
    tx_to_gene_matrix[tx_ind[tx_key],gene_ind[tx_to_gene[tx_key]]]=1
print(len(gene_ind))
print(len(tx_ind))
print(len(tx_to_gene))
#meta_df=pd.read_csv(filepath_or_buffer=metadata_file,sep="\t")   
iter_meta = pd.read_csv(filepath_or_buffer=metadata_file,sep="\t", iterator=True, chunksize=1000)

#meta_df = pd.concat([chunk[chunk[geo_acc_name].str.startswith('GSM')] for chunk in iter_meta])
meta_df = pd.concat([chunk for chunk in iter_meta])
true_srr=len(meta_df)
print(true_srr)
meta_df=meta_df[meta_df[geo_acc_name].str.startswith('GSM')]
n_srr= len(meta_df)
print(n_srr)
srr_to_gsm= dict(zip(meta_df["SRR_accession"],meta_df[geo_acc_name]))
srr_ind= {srr:ind for ind,srr in enumerate(srr_to_gsm.keys())}
gsm_ind= {gsm:ind for ind,gsm in enumerate(pd.unique(meta_df[geo_acc_name]))}
gsm_from_srr_matrix= sps.lil_matrix((len(gsm_ind),len(srr_ind)))
for srr_key in srr_to_gsm:
    gsm_from_srr_matrix[gsm_ind[srr_to_gsm[srr_key]],srr_ind[srr_key]]=1
print(gsm_from_srr_matrix[0,:])


FileNotFoundError: File b'D:/datasets/DEE2/TxInfo/hsapiens_TxInfo.tsv' does not exist

Read dee2,transform via sparse matrices and write to h5

In [43]:
%%time
import datetime
from scipy.sparse import isspmatrix_csr,csr_matrix,csc_matrix
data_file_name= cache_dir+"raw_files/"+organism_name+"_ke.tsv.bz2"
h5_gsm_name = cache_dir+organism_name+ "_kalisto_matrix.h5"
now = datetime.datetime.now()
with h5py.File(h5_gsm_name, 'w') as h5_gse:
    meta_grp = h5_gse.create_group('meta')
    info_grp = h5_gse.create_group('info')
    data_grp= h5_gse.create_group('data')
    info_grp.create_dataset('version', data="dee2_gse_v1")
    info_grp.create_dataset('creation_date', data=now.strftime("%Y.%m.%d"))
    n_genes =len(gene_ind)
    dt = h5py.special_dtype(vlen=np.str)
    meta_grp.create_dataset('genes',data=gene_ind.keys(),dtype=dt)
    n_gsm=len(gsm_ind)        
    meta_grp.create_dataset('Sample_geo_accession',data=gsm_ind.keys(), dtype =dt)
    meta_grp.create_dataset('Sample_instrument_model',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_last_update_date',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_library_selection',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_library_source',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_library_strategy',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_organism_ch1',(n_gsm,),dtype =dt)
    meta_grp.create_dataset('Sample_series_id',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_status',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_submission_date',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_taxid_ch1',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_quality',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_title',(n_gsm,), dtype =dt)
    meta_grp.create_dataset('Sample_type',data=np.full(n_gsm,"SRA", dtype =dt))
    exp_data=data_grp.create_dataset("expression", (n_gsm,n_genes),dtype= 'i4')#, compression="gzip", compression_opts=9)
    processed_gse=0
    srr_per_time=1000
    row_length= len(tx_ind)
    iter_data = pd.read_csv(filepath_or_buffer=data_file_name,sep="\t", iterator=True, chunksize=srr_per_time*row_length,names=["srr","tx","ke"])
    proc_srr= 0
    gsm_from_srr_matrix=csr_matrix(gsm_from_srr_matrix)    
    tx_to_gene_matrix=csr_matrix(tx_to_gene_matrix)
    for chunk in iter_data:
        global_ids =[]
        local_ids=[]
        local_srr_ind= {srr:ind for ind,srr in enumerate(pd.unique(chunk['srr']))}
        srr_count=len(local_srr_ind)
        proc_srr=proc_srr+srr_count
        for cur_srr in local_srr_ind:
            try:
                srr_id= srr_ind[cur_srr]
                global_ids.append(srr_id)
                local_ids.append(local_srr_ind[cur_srr])
            except(KeyError):
                continue
        if local_ids:           
            raw_matrix= chunk["ke"].values.astype(int).reshape(srr_count,row_length)
            A=gsm_from_srr_matrix[:,global_ids]
            gsm_mask=[np.sum(x)>0 for x in A]            
            A = A.tocsr()
            A = A.dot(raw_matrix[local_ids,])
            A=csr_matrix(A)
            A = A.dot(tx_to_gene_matrix)
            exp_data[gsm_mask== True,0:n_genes]= exp_data[gsm_mask== True,0:n_genes]+A[gsm_mask== True,0:n_genes]
        if proc_srr%500 ==0:
            print(proc_srr/true_srr)
    h5_gse.flush()
print(1)

  

0.14236902050113895
0.2847380410022779
0.4271070615034169
0.5694760820045558
0.7118451025056948
0.8542141230068337
0.9965831435079726
1
Wall time: 40.7 s


In [19]:
a=np.array([1,2,3,4,5,6,7,8,9]).reshape(3,3)
a
for i in a:
    print(np.sum(i))
b=np.matrix('1 2; 3 4')
for i in b:
    print(np.sum(i))

6
15
24
3
7


dee2 kalisto to h5 files

In [89]:
%%time
import bz2
import h5py
import numpy as np
import pandas as pd
import scipy.sparse as sps
import datetime
import math
cache_dir="D:/datasets/DEE2/"
organism_name= 'ecoli'
tx_info_file = cache_dir+"TxInfo/"+ organism_name+"_TxInfo.tsv"
metadata_file = cache_dir+"metadata/"+ organism_name+"_metadata.tsv"
data_file_name= cache_dir+"raw_files/"+organism_name+"_ke.tsv.bz2"
h5_gsm_name = cache_dir+organism_name+ "_kalisto_matrix.h5"
#check txs
#tx_dict=dict()
#with open(tx_info_file, 'r') as info_file:
#    for line in info_file:
#        tx_id, gene_id, *tail= line.split("\t")
#        try:
#            if (tx_dict[tx_id] != gene_id):
#                print("Error. tx in few genes")
#                break
#        except(KeyError):
#            tx_dict[tx_id] = gene_id
#
#            
#print(len(tx_dict))


tx_info=pd.read_csv(filepath_or_buffer=tx_info_file,sep="\t")
tx_to_gene= dict(zip(tx_info["TxID"],tx_info["GeneID"]))
genes= pd.unique(tx_info["GeneID"])
genes=dict(zip(genes,range(0,len(genes))))
tx_g_shape = (len(tx_to_gene), len(genes))
tx_to_gene_matrix= sps.coo_matrix(tx_g_shape)
for (tx_key in tx_to_gene):
    tx_to_gene_matrix[]

meta_df=pd.read_csv(filepath_or_buffer=metadata_file,sep="\t")
n_srr= len(meta_df)
srr_to_gsm= dict(zip(meta_df["SRR_accession"],meta_df["GSE_accession"]))
gsms= pd.unique([x for x in meta_df["GSE_accession"] if ('GSM' in x)])
gsms= dict(zip(gsms,range(0,len(gsms))))





with h5py.File(h5_gsm_name, 'w') as h5_gse:
        meta_grp = h5_gse.create_group('meta')
        info_grp = h5_gse.create_group('info')
        data_grp= h5_gse.create_group('data')
        info_grp.create_dataset('version', data="dee2_gse_v1")
        info_grp.create_dataset('creation_date', data=now.strftime("%Y.%m.%d"))
        n_genes =len(genes)
        dt = h5py.special_dtype(vlen=np.str)
        meta_grp.create_dataset('genes',data=genes.keys(),dtype=dt)
        n_gsm=len(gsms)        
        meta_grp.create_dataset('Sample_geo_accession',data=gsms.keys(), dtype =dt)
        meta_grp.create_dataset('Sample_instrument_model',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_last_update_date',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_library_selection',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_library_source',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_library_strategy',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_organism_ch1',(n_gsm,),dtype =dt)
        meta_grp.create_dataset('Sample_series_id',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_status',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_submission_date',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_taxid_ch1',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_quality',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_title',(n_gsm,), dtype =dt)
        meta_grp.create_dataset('Sample_type',data=np.full(n_gsm,"SRA", dtype =dt))
        exp_data=data_grp.create_dataset("expression", (n_gsm,n_genes),dtype= 'i4')#, compression="gzip", compression_opts=9)
        processed_gse=0
        with bz2.open(data_file_name, 'rt') as data_file:
            proc_srr=0
            n_srr= 30357728
            for line in data_file:
                srr,tx,ke=line.split("\t")
                proc_srr+=1
                if(proc_srr%300000==0):
                    print(proc_srr/n_srr)
                cur_gsm=srr_to_gsm[srr]
                try:
                    gsm_ind=gsms[cur_gsm]
                    cur_gene=tx_to_gene[tx]
                    gene_ind=genes[cur_gene]
                    exp_data[gsm_ind,gene_ind]=exp_data[gsm_ind,gene_ind]+math.floor(float(ke))    
                except(KeyError):
                    continue
                            
                
        h5_gse.flush()

0.009882162459588544
0.01976432491917709
0.029646487378765633
0.03952864983835418
0.04941081229794272
0.059292974757531265
0.0691751372171198
0.07905729967670835
0.08893946213629689
0.09882162459588544
0.10870378705547398
0.11858594951506253
0.12846811197465108
0.1383502744342396
0.14823243689382815
0.1581145993534167
0.16799676181300524
0.17787892427259377
0.18776108673218234
0.19764324919177087
0.2075254116513594
0.21740757411094797
0.2272897365705365
0.23717189903012506
0.2470540614897136
0.25693622394930216
0.26681838640889066
0.2767005488684792
0.2865827113280678
0.2964648737876563
0.30634703624724485


KeyboardInterrupt: 

In [75]:
gsms= pd.unique([x for x in meta_df["GSE_accession"] if ('GSM' in x)])
print(len(gsms))
print(type(genes))
print(type(gsms))
gsms= dict(zip(gsms,range(0,len(gsms))))
#print(gsms.keys())
print(len(meta_df))
genes=dict(zip(genes,range(0,len(genes))))

meta_df["SampleName"]
float('3.1111\n')

3193
<class 'dict'>
<class 'numpy.ndarray'>
7024


3.1111

In [53]:

d= defaultdict(list)
d["qwe"].append(1)
d["qwe"].append(10)
d["ry"].append(2)
print(list(d.keys()))

['qwe', 'ry']


In [14]:

reader = pd.read_csv("D:/datasets/test.txt", sep='\t',names=["digit","qwe"], iterator=True)

print(reader.get_chunk(5))
print(reader.get_chunk(10))
print(reader)
print([1,2,3,4,5,5,6,78,8][1:4])

   digit  qwe
0    111  aaa
1    222  bbb
2    333  ccc
3    444  ddd
4    555  eee
   digit  qwe
5    666  fff
6    777  ggg
7    888  hhh
8    999  iii
[2, 3, 4]


In [10]:
arch_file = "D:/datasets/ARCHS4_v7/human_matrix.h5"
with h5py.File(arch_file, 'a') as arch_7:
    print("сетов в матрице " + str(len(arch_7["data"]["expression"])))
    print( "генов в матрице " + str(len(arch_7["data"]["expression"][1])))
    for key in arch_7["meta"].keys():
        print(key)
    print(len(arch_7["meta"]["genes"]))

сетов в матрице 167726
генов в матрице 35238
Sample_channel_count
Sample_characteristics_ch1
Sample_contact_address
Sample_contact_city
Sample_contact_country
Sample_contact_department
Sample_contact_email
Sample_contact_institute
Sample_contact_laboratory
Sample_contact_name
Sample_contact_phone
Sample_contact_zip-postal_code
Sample_data_processing
Sample_data_row_count
Sample_description
Sample_extract_protocol_ch1
Sample_geo_accession
Sample_instrument_model
Sample_last_update_date
Sample_library_selection
Sample_library_source
Sample_library_strategy
Sample_molecule_ch1
Sample_organism_ch1
Sample_platform_id
Sample_relation
Sample_series_id
Sample_source_name_ch1
Sample_status
Sample_submission_date
Sample_supplementary_file_1
Sample_supplementary_file_2
Sample_taxid_ch1
Sample_title
Sample_type
gene_accession
gene_chromosome
gene_entrezid
gene_hgnc
gene_name
gene_refseqid
gene_synonym
genes
reads_aligned
total_reads
35238


In [9]:
arch_file_8 = "D:/datasets/phantasus_cache/archs4/human_matrix_8.h5"
with h5py.File(arch_file_8, 'a') as arch_8:
    print("сетов в матрице " + str(len(arch_8["data"]["expression"])))
    print( "генов в матрице " + str(len(arch_8["data"]["expression"][1])))
    for key in arch_8["meta"].keys():
        print(key)
    print(len(arch_8["meta"]["genes"]))

сетов в матрице 238522
генов в матрице 35238
Sample_channel_count
Sample_characteristics_ch1
Sample_contact_address
Sample_contact_city
Sample_contact_country
Sample_contact_department
Sample_contact_email
Sample_contact_institute
Sample_contact_laboratory
Sample_contact_name
Sample_contact_phone
Sample_contact_zip-postal_code
Sample_data_processing
Sample_data_row_count
Sample_description
Sample_extract_protocol_ch1
Sample_geo_accession
Sample_instrument_model
Sample_last_update_date
Sample_library_selection
Sample_library_source
Sample_library_strategy
Sample_molecule_ch1
Sample_organism_ch1
Sample_platform_id
Sample_relation
Sample_series_id
Sample_source_name_ch1
Sample_status
Sample_submission_date
Sample_supplementary_file_1
Sample_supplementary_file_2
Sample_taxid_ch1
Sample_title
Sample_type
gene_accession
gene_chromosome
gene_entrezid
gene_hgnc
gene_name
gene_refseqid
gene_synonym
genes
reads_aligned
reads_total
35238


In [17]:
arch_7= h5py.File(arch_file, 'a')
arch_8= h5py.File(arch_file_8, 'a')

arch_7["meta"].keys()[arch_8["meta"].keys()!= arch_7["meta"].keys()]

TypeError: 'KeysView' object does not support indexing

In [1]:
import bz2